# run this first to set up the libraries

In [9]:
import os
os.chdir('/src/notebooks')
import lib.postgres as psql
con, cur = psql.connect_to_postgres()
from redis import Redis
from rq import Queue
import lib.playgames as play
REDIS = Redis(host='this_redis')
Q = Queue(connection=REDIS)

##############################################################################################
# this function was working as a redis job but something broke, ill try to fix it in an update
##############################################################################################
import chess
import chess.engine
import chess.svg
import chess.pgn
import chess.polyglot
import lib.postgres as psql
import io
import os
def playGame( whiteName,blackName,timePerMove,line ):
    os.chdir('/src/notebooks')
    if whiteName == 'leela':
        white = chess.engine.SimpleEngine.popen_uci("/lczero/lc0")
        black = chess.engine.SimpleEngine.popen_uci("/root/Stockfish/src/stockfish")
    else:
        black = chess.engine.SimpleEngine.popen_uci("/lczero/lc0")
        white = chess.engine.SimpleEngine.popen_uci("/root/Stockfish/src/stockfish")
    board = chess.Board()
    openingGame = chess.pgn.read_game(io.StringIO(line))
    game = chess.pgn.Game()

    for move in openingGame.mainline_moves():
        if ((board.fullmove_number == 1) & board.turn):
            node = game.add_variation(chess.Move.from_uci(str(move)))
        else:
            node = node.add_variation(chess.Move.from_uci(str(move)))
        board.push(chess.Move.from_uci(str(move)))
        
    while not board.is_game_over(claim_draw=True):
        if board.turn:
            result = white.play(board, chess.engine.Limit(time=timePerMove))
        else:
            result = black.play(board, chess.engine.Limit(time=timePerMove))

        node = node.add_variation(result.move)
        board.push(result.move)

    game.headers["Result"] = board.result()
    game.headers["White"] = whiteName
    game.headers["Black"] = blackName
    psql.insert_game(game,timePerMove)

# suppose you want to see leela and stockfish play the King's gambit, just put the opening moves in here and run the playGame function...

In [10]:
line = "1. e4 e5 2. d4"
playGame("leela","stockfish",1,line)

<UciProtocol (pid=846)>: stderr >>        _
<UciProtocol (pid=846)>: stderr >> |   _ | |
<UciProtocol (pid=846)>: stderr >> |_ |_ |_| v0.21.5-rc1 built May  2 2021
<UciProtocol (pid=846)>: stderr >> Found pb network file: /lczero/J20-460
<UciProtocol (pid=846)>: stderr >> Creating backend [cudnn]...
<UciProtocol (pid=846)>: stderr >> GPU: Tesla V100-PCIE-32GB
<UciProtocol (pid=846)>: stderr >> GPU memory: 31.7485 Gb
<UciProtocol (pid=846)>: stderr >> GPU clock frequency: 1380 MHz
<UciProtocol (pid=846)>: stderr >> GPU compute capability: 7.0
<UciProtocol (pid=846)>: stderr >> CUDA Runtime version: 10.0.0
<UciProtocol (pid=846)>: stderr >> Cudnn version: 7.6.5
<UciProtocol (pid=846)>: stderr >> Latest version of CUDA supported by the driver: 10.2.0
<UciProtocol (pid=846)>: stderr >> WARNING: you will probably get better performance from the cudnn-fp16 backend.


# After you launch the games you can go to 127.0.0.1:5000 or something similar to see job running. (having trouble all of a sudden running the games as background jobs, ill look into the issue and commit when I fix it)

In [11]:
cur.execute("select * from games order by game desc limit 8")
games = cur.fetchall()
for game in games:
    print(game)
    print("\n")

[1, 'leela', 'stockfish', '*', 1, datetime.datetime(2021, 5, 2, 17, 34, 20, 276156), '1. e4 e5 2. d4 exd4 3. Nf3 Bb4+ 4. Bd2 Bxd2+ 5. Qxd2 c5 6. c3 Nf6 7. e5 Ne4 8. Qf4 d5 9. exd6 O-O 10. Qxe4 Re8 11. Ne5 Nc6 12. cxd4 cxd4 13. f4 Nxe5 14. fxe5 Qg5 15. Qxd4 Rxe5+ 16. Kf2 Rd5 17. Qc3 Bd7 18. Na3 Rd2+ 19. Be2 Rxe2+ 20. Kxe2 Qxg2+ 21. Kd3 Bf5+ 22. Kc4 Rc8+ 23. Kb3 Rxc3+ 24. bxc3 Qd5+ 25. Kb2 Qd2+ 26. Kb3 Qd5+ 27. Kb2 Qd2+ 28. Kb3']


